# Day 11 Lab

*Adapted from Bioinformatics Algorithms Chapter 4.*

## Part B - De Novo Peptide Sequencing

As it turns out, Tyrocidine is a non-ribosomal protein. This means there are no segments of DNA that encode for Tyrocidine.

To figure out the sequence of a peptide like Tyrocidine, we need to use [Mass Spectrometry](https://www.bioinfor.com/denovo-tutorial/).

This is going to break the peptide down into fragments and the masses of each fragment are going to be measured. 

We are now going to use a branch and bound algorithm to determine possible peptide sequences from a spectrum. 

Before we write our algorithm, I want you to write a bit about the algorithm and then we will write some helper functions. 

#### (1) What is a branch and bound algorithm? Are there key steps to these kinds of algorithms?
*Practice writing about the algorithm here.*

Algorithm that is an exhaustive search to solve optimization problems, with a finite set of possibilities.
1) Initialization
2) Branching
3) Bounding
4) Check
5) Exploration

#### (2) Since our mass spectrum output is going to be a set of masses, we are going to think of our peptides as strings of masses, instead of strings of characters. Write a function to convert a peptide to a string of masses.

`PeptideToMass("NEQL")` would output `'114-129-128-113'`

To help, here is a dictionary of amino acid masses. Note that we are going to assume integer masses and that there are only 18 distinct masses.

In [1]:
aaMasses = {'G': 57, 'A': 71, 'S': 87, 'P': 97, 'V': 99, 'T': 101, 'C': 103, 'I': 113, 'L': 113, 'N': 114, 'D': 115, 'K': 128, 'Q': 128, 'E': 129, 'M': 131, 'H': 137, 'F': 147, 'R': 156, 'Y': 163, 'W': 186}

In [2]:
def peptideToMax(sequence):
    l = len(sequence)
    seqList = []
    for i in range(l):
        seqList.append(aaMasses[sequence[i]])
    finalString = ""
    for j in range(l):
        finalString+=str(seqList[j])
        if j!=l-1:
            finalString+="-"
    return finalString
print(peptideToMax("NEQL"))

114-129-128-113


#### (3) It might be useful to have a function that goes the other way to. Write a `MassToPeptide` function.

`MassToPeptide('114-129-128-113')` would output `"NE(K/Q)(I/L)"`

*Note: I (Isoleucine) and L (Leucine) have the same mass. We cannot determine which is the correct peptide for the last position. Both should be included.* 

In [3]:
massesAA = {57: 'G', 71: 'A', 87: 'S', 97: 'P', 99: 'V', 101: 'T', 103: 'C', 113: '(I/L)', 114: 'N', 115: 'D', 128: '(K/Q)', 129: 'E', 131: 'M', 137: 'H', 147: 'F', 156: 'R', 163: 'Y', 186: 'W'}

In [4]:
def massToPeptide(mass: str):
    pepList = mass.split('-')
    newList = []
    for entry in pepList:
        newList.append(int(entry))
    final = ""
    for number in newList:
        final+=massesAA[number]
    return final
print(massToPeptide('114-129-128-113'))

NE(K/Q)(I/L)


#### (4) Given a peptide, we need to know values would exist in its spectrum. This will help us during the bounding phase of our algorithm. Write a `LinearSpectrum` function that will convert a peptide to a list of masses that would exist in its spectrum.

`LinearSpectrum("NQEL")` would yield `[0, 113, 114, 128, 129, 242, 242, 257, 370, 371, 484]`



<img src = "https://bioinformaticsalgorithms.com/images/Antibiotics/NQEL_linear_spectrum.png" width = 500>

In [5]:
def linearSpectrum(sequence):
    prefixMass = [aaMasses[sequence[0]]]
    l = len(sequence)
    
    for i in range(1,l):
        prefixMass.append(prefixMass[i-1] + aaMasses[sequence[i]])
    linSpectrum = [0]
    i = 0
    for i in range(l-1):
        for j in range(i+1,l):
            linSpectrum.append(prefixMass[j]-prefixMass[i])
    for entry in prefixMass:
        linSpectrum.append(entry)
    linSpectrum.sort()
    return linSpectrum
    
print(linearSpectrum("NQEL"))
    

[0, 113, 114, 128, 129, 242, 242, 257, 370, 371, 484]


***
Our LinearSpectrum function was a great start, but recall that the peptides we are looking at are cyclical. The function above misses all peptides at the overlap. For example, if we had `NEQL` the above function misses the peptide `LN`, `ELN` and `LNQ` among others.

#### (5) Edit your `LinearSpectrum` to take into account the cyclic nature of our peptides. 
`CyclicSpectrum("NQEL")` would yield an output of `[0, 113, 114, 128, 129, 227, 242, 242, 257, 355, 356, 370, 371, 484]`

<img src = "https://bioinformaticsalgorithms.com/images/Antibiotics/duplicate_elements.png" width = 500>

*Note: The changes to the pseudocode are lines 7, 12 and 13.*

In [6]:
def cyclicSpectrum(sequence):
    prefixMass = [aaMasses[sequence[0]]]
    l = len(sequence)
    
    for i in range(1,l):
        prefixMass.append(prefixMass[i-1] + aaMasses[sequence[i]])
        
    parentMass = prefixMass[l-1]
    cycSpectrum = [0]
    for i in range(l-1):
        for j in range(i+1,l):
            cycSpectrum.append(prefixMass[j] - prefixMass[i])
            if i < j and j < l-1:
                cycSpectrum.append(parentMass-(prefixMass[j] - prefixMass[i]))
    for entry in prefixMass:
        cycSpectrum.append(entry)
    cycSpectrum.sort()
    return cycSpectrum
print(cyclicSpectrum("NQEL"))

[0, 113, 114, 128, 129, 227, 242, 242, 257, 355, 356, 370, 371, 484]


#### (6) Edit your `LinearSpectrum` and `CyclicSpectrum` to work with a peptide in mass form. 
`LinearSpectrumMass('114-128-129-113')` would yield `[0, 113, 114, 128, 129, 242, 242, 257, 370, 371, 484]`
`CyclicSpectrumMass('114-128-129-113')` would yield `[0, 113, 114, 128, 129, 227, 242, 242, 257, 355, 356, 370, 371, 484]`

In [7]:
def superMassToPeptide(massList):
    peptide = massToPeptide(massList)
    final = ""
    i = 0
    while i<len(peptide):
        if peptide[i] == "(":
            final+=peptide[i+1]
            i+=5
        else:
            final+=peptide[i]
            i+=1
    return final
def linearSpectrumMass(massList):
    return linearSpectrum(superMassToPeptide(massList))
def cyclicSpectrumMass(massList):
    return cyclicSpectrum(superMassToPeptide(massList))
x = '114-128-129-113'
print(linearSpectrumMass(x))
print(cyclicSpectrumMass(x))

[0, 113, 114, 128, 129, 242, 242, 257, 370, 371, 484]
[0, 113, 114, 128, 129, 227, 242, 242, 257, 355, 356, 370, 371, 484]


#### (7) Write a function, `MassInSpectrum(spectrum, aminoAcidMasses = aaMasses)` to determine which individual amino acid masses exist in a spectrum. 

`MassInSpectrum([0,113,128,186,241,299,314,427])` would return `{113,128,186}`

*This function will help us narrow our search field even more when we get to the branching step of our branch and bound algorithm. There is no need to include individual masses in the branch step that don't appear in original spectrum. For a peptide like NEQL, we would only need to have 4 branches at each step instead of 18.*

In [8]:
def massInSpectrum(spectrum):
    return set([mass for mass in spectrum if mass in aaMasses.values()])
print(massInSpectrum([0,113,128,186,241,299,314,427]))

{128, 113, 186}


#### (8) Next, write a function called `ExpandPeptides(peptideMassesLst, masses)` that will take in a list of peptides in mass form and a set of possible masses to add on. This function will be called during the branching step of our algorithm.

`ExpandPeptides(['114-129', '113-129'],{113,114,128,129})` would yield `['114-129-128', '114-129-113', '114-129-114', '114-129-129', '113-129-128', '113-129-113', '113-129-114', '113-129-129']`

In [17]:
def expandPeptides(peptideMassesList, masses):
    finalList = []
    for entry in peptideMassesList:
        for set in masses:
            finalList.append(entry+'-'+str(set))
    return finalList
print(expandPeptides(['114-129', '113-129'],{113,114,128,129}))

['114-129-128', '114-129-113', '114-129-114', '114-129-129', '113-129-128', '113-129-113', '113-129-114', '113-129-129']


#### (9) Write a function that will take the sum of masses from a peptide in mass form.

`Mass('114-129-114')` would yield `357`

In [15]:
def mass(masses):
    massList = []
    masses = masses + ' '
    i=0
    l=len(masses)
    while i < l:
        if masses[i]=="-" or masses[i]==' ':
            massList.append(int(masses[i-3:i]))
        i+=1
    return(sum(massList))
print(mass('114-129-114'))
        

357


#### (10) Finally, we are ready to write our branch and bound algorithm! Define the `CyclopeptideSequencing(spectrum)`.

`CyclopeptideSequencing([0, 113, 128, 186, 241, 299, 314, 427])` would yield a list like `['186-128-113', '186-113-128', '128-186-113', '128-113-186', '113-186-128', '113-128-186']`

In [21]:
def cyclopeptideSequencing(spectrum):
    aaCandidateMasses = massInSpectrum(spectrum)
    candidatePeptides = [str(x) for x in aaCandidateMasses]
    parentMass = max(spectrum)
    finalPeptides = []
    
    while candidatePeptides:
        candidatePeptides: list = expandPeptides(candidatePeptides, aaCandidateMasses)
        
        i=0
        while i < len(candidatePeptides):
            peptide = candidatePeptides[i]
            pepMass = mass(peptide)
            if pepMass == parentMass:
                if cyclicSpectrumMass(peptide) == spectrum:
                    finalPeptides.append(peptide)
                candidatePeptides.remove(peptide)
            elif pepMass > parentMass:
                candidatePeptides.remove(peptide)
            elif not all([massA in spectrum for massA in linearSpectrumMass(peptide)]):
                candidatePeptides.remove(peptide)
            else:
                i += 1
    return finalPeptides
print(cyclopeptideSequencing([0, 113, 128, 186, 241, 299, 314, 427]))


['128-113-186', '128-186-113', '113-128-186', '113-186-128', '186-128-113', '186-113-128']
